In [7]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [9]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [11]:
%sql sqlite:///my_data1.db

In [13]:
import pandas as pd

df = pd.read_csv("Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [15]:
%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [17]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

the names of the unique launch sites  in the space mission

In [29]:
%%sql
SELECT DISTINCT Launch_Site 
FROM SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


Display 5 records where launch sites begin with the string 'CCA' 

In [35]:
%%sql
SELECT *
FROM SPACEXTABLE
WHERE "Launch_Site" LIKE "CCA%"
LIMIT 5
;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [41]:
%%sql

SELECT SUM(PAYLOAD_MASS__KG_) AS Total_PayloadMass
FROM SPACEXTABLE
WHERE Customer LIKE 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


Total_PayloadMass
45596


In [47]:
%%sql

SELECT AVG(PAYLOAD_MASS__KG_) AS Avg_PayloadMass
FROM SPACEXTABLE
WHERE Booster_Version = 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


Avg_PayloadMass
2928.4


In [49]:
%%sql
SELECT MIN(Date) AS FirstSuccessfull_landing_date
FROM SPACEXTABLE
WHERE Landing_Outcome LIKE 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


FirstSuccessfull_landing_date
2015-12-22


In [53]:
%%sql

SELECT Booster_Version
FROM SPACEXTABLE
WHERE Landing_Outcome = 'Success (drone ship)'
AND PAYLOAD_MASS__KG_ > 4000 
AND PAYLOAD_MASS__KG_ < 6000

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [55]:
%%sql

SELECT COUNT(Mission_Outcome) AS SuccessOutcome
FROM SPACEXTABLE
WHERE Mission_Outcome LIKE 'Success%'

 * sqlite:///my_data1.db
Done.


SuccessOutcome
100


In [57]:
%%sql
SELECT COUNT(Mission_Outcome) AS FailureOutcome
FROM SPACEXTABLE
WHERE Mission_Outcome LIKE 'Failure%'

 * sqlite:///my_data1.db
Done.


FailureOutcome
1


In [61]:
%%sql

SELECT Booster_Version, PAYLOAD_MASS__KG_
FROM SPACEXTABLE
WHERE PAYLOAD_MASS__KG_ = (
                        SELECT MAX(PAYLOAD_MASS__KG_)
                        FROM SPACEXTABLE
                        )
ORDER BY Booster_Version

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1048.5,15600
F9 B5 B1049.4,15600
F9 B5 B1049.5,15600
F9 B5 B1049.7,15600
F9 B5 B1051.3,15600
F9 B5 B1051.4,15600
F9 B5 B1051.6,15600
F9 B5 B1056.4,15600
F9 B5 B1058.3,15600


In [63]:
%%sql

SELECT Booster_Version, Launch_Site, Landing_Outcome
FROM SPACEXTABLE
WHERE Landing_Outcome LIKE 'Failure (drone ship)'
AND Date BETWEEN '2015-01-01' AND '2015-12-31'

 * sqlite:///my_data1.db
Done.


Booster_Version,Launch_Site,Landing_Outcome
F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


In [65]:
%%sql

SELECT Landing_Outcome, COUNT(Landing_Outcome)
FROM SPACEXTABLE
WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY COUNT(Landing_Outcome) DESC

 * sqlite:///my_data1.db
Done.


Landing_Outcome,COUNT(Landing_Outcome)
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
